# Data Cleaning & Wrangling
### <font color='burgundy'> MinneMUDAC Data Science Challenge, Fall 2018</font>

This is the data cleaning/wrangling portion of the MinneMUDAC Data Science Project, Fall 2018. Please sear readme for link to project description. Something to keep in mind is that we have congressional elections every two years, but gubernatorial and senate elections happen less often (so missing data for those elections will indicate that there was no election that year)

First let us start with some packages we will need throughout.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import imp
import numpy as np
import sklearn

Next, we look at the election results for the years 1994 to 2016 (every two). One problem we notice is that some of these years (1996 and 1998) do not include the congressional district, so we try to match these using county code, precinct code or name. Because of redrawing of the districts, it is best we use a year close to these years. 1994 contains congressional district information, so we'll use that.

In [2]:
# Ignore warnings. Careful with this one.
import warnings
warnings.simplefilter('ignore')

In [3]:
# Read data
df94=pd.read_csv("../Data sets/1994_results.csv")
df96=pd.read_csv("../Data sets/1996_results.csv")
df98=pd.read_csv("../Data sets/1998_results.csv")
df00=pd.read_csv("../Data sets/2000_results.csv")
df02=pd.read_csv("../Data sets/2002_results.csv")
df04=pd.read_csv("../Data sets/2004_results.csv")
df06=pd.read_csv("../Data sets/2006_results.csv")
df08=pd.read_csv("../Data sets/2008_results.csv")
df10=pd.read_csv("../Data sets/2010_results.csv")
df12=pd.read_csv("../Data sets/2012_results.csv")
df14=pd.read_csv("../Data sets/2014_results.csv")
df16=pd.read_csv("../Data sets/2016_results.csv")

# We can look at column names as follows. Note that we also made a key for all of the column names.
list(df94.columns.values)

['FIPS_VTD',
 'Precinct Name',
 'CC',
 'PRCT',
 'CG',
 'LEG',
 'CM',
 '7am',
 'EDR',
 'Signatures',
 'AB-Reg',
 'AB-Fed',
 'AB-Pres',
 'Ballots',
 'USSenIR',
 'USSenDFL',
 'USSenOther',
 'CongIR',
 'CongDFL',
 'CongOther',
 'MNLegIR',
 'MNLegDFL',
 'MNLegOther',
 'GovIR',
 'GovDFL',
 'GovOther',
 'SOSIR',
 'SOSDFL',
 'SOSOther',
 'AGIR',
 'AGDFL',
 'AGOther',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37']

In [4]:
# After a quick search of the 1994 data, we see that county code and precinct code are not the
# best choice because they might change throughout the years. Let us try searching by name.

# Add empty columns for congressional district
df96['CG']=np.nan
df98['CG']=np.nan

import string

translator = str.maketrans('', '', string.punctuation)

# df94['PCTNAME']=df94["PCTNAME"].str.upper() #Also convert precinct name to uppercase

#remove all punctuation and white space
df94['Precinct Name']=df94['Precinct Name'].str.translate(translator).str.replace(" ","")    
df96['Precinct Name']=df96['Precinct Name'].str.translate(translator).str.replace(" ","")
df98['Precinct Name']=df98['Precinct Name'].str.translate(translator).str.replace(" ","")


# Next, let us re-fill the congressional district entries using the precinct names
for i in range(0,df96.shape[0]):
    temp=df94[df94['Precinct Name'].str.contains(df96.loc[i,'Precinct Name'])]
    if temp.size!=0: #make sure we actually found the precinct
        df96.loc[i,'CG']=temp.iloc[0,:]['CG']

for i in range(0, df98.shape[0]):
    # For each row, we locate the precinct in the 1994 data set to find its cong. district
    temp=df94[df94['Precinct Name'].str.contains(df98.loc[i,'Precinct Name'])]
    if temp.size!=0: #make sure we actually found the precinct
        df98.loc[i,'CG']=temp.iloc[0,:]['CG']
        
# 92 is still somewhat tricky. Come back to this later.

        
print('Number of empty CG entries for 1998: ',df98.isna().sum().CG)
print('Number of empty CG entries for 1996: ',df96.isna().sum().CG)

Number of empty CG entries for 1998:  107
Number of empty CG entries for 1996:  63


In [5]:
# Good, we've got the number of empty entries down enough to fill in by hand.
# Let us save these

df96.to_csv('df96.csv')
df98.to_csv('df98.csv')

### IPUMS Data
Next, we deal with the census data from IPUMS. We start off with some quick data exploration to get a sense of what we have.

In [6]:
df=pd.read_csv('../Data sets/ipums_extract1.csv')
df.head()

,YEAR,DATANUM,SERIAL,HHWT,REGION,STATEICP,STATEFIP,COUNTY,COUNTYFIPS,URBAN,...,HCOVANY,HIGRADE,HIGRADED,EDUC,EDUCD,EMPSTAT,EMPSTATD,OCC,IND,INCTOT
0,1850,1,1262501,99.29,22,33,27,90.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
1,1850,1,1262501,99.29,22,33,27,90.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,NaN,NaN
2,1850,1,1262501,99.29,22,33,27,90.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3,1850,1,1262501,99.29,22,33,27,90.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999.0,NaN,NaN
4,1850,1,1262601,99.29,22,33,27,610.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN


In [7]:
print(df.shape)
print(df.columns.values)
print(df.YEAR.unique())

(2044399, 32)
['YEAR' 'DATANUM' 'SERIAL' 'HHWT' 'REGION' 'STATEICP' 'STATEFIP' 'COUNTY'
 'COUNTYFIPS' 'URBAN' 'METRO' 'CITYPOP' 'GQ' 'PERNUM' 'PERWT' 'SEX' 'AGE'
 'MARST' 'RACE' 'RACED' 'HISPAN' 'HISPAND' 'HCOVANY' 'HIGRADE' 'HIGRADED'
 'EDUC' 'EDUCD' 'EMPSTAT' 'EMPSTATD' 'OCC' 'IND' 'INCTOT']
[1850 1860 1870 1880 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016]


Observe that our general voter turnout data goes all the way back to 1950 but our results data only goes back to 1992, 
while the census data contains only 1990 from the 90's.. Perhaps we can use 1990 to get data for 1992-1998?

Let us do a little bit more exploration.

In [8]:
# Let us start by counting how many entries we have for each year as well as the missing county data.
c=[];
for i in range(1850,2010,10):
    temp=df[df['YEAR']==i]
    c.append(temp.shape[0])
    print('-',temp.shape[0], "entries for year",i,"with",temp.isna().sum().COUNTY, "missing county entries")

for i in range(2001,2017):
    temp=df[df['YEAR']==i]
    c.append(temp.shape[0])
    print('-',temp.shape[0], "entries for year",i,"with",temp.isna().sum().COUNTY, "missing county entries")

- 58 entries for year 1850 with 0 missing county entries
- 1706 entries for year 1860 with 0 missing county entries
- 4633 entries for year 1870 with 0 missing county entries
- 79030 entries for year 1880 with 0 missing county entries
- 0 entries for year 1890 with 0 missing county entries
- 89352 entries for year 1900 with 0 missing county entries
- 20821 entries for year 1910 with 0 missing county entries
- 23990 entries for year 1920 with 0 missing county entries
- 127442 entries for year 1930 with 0 missing county entries
- 28725 entries for year 1940 with 0 missing county entries
- 37832 entries for year 1950 with 0 missing county entries
- 170578 entries for year 1960 with 0 missing county entries
- 56137 entries for year 1970 with 0 missing county entries
- 203721 entries for year 1980 with 0 missing county entries
- 226174 entries for year 1990 with 0 missing county entries
- 249237 entries for year 2000 with 0 missing county entries
- 21188 entries for year 2001 with 21188 mis

Unfortunately, years 2001-2004 have many missing county entries.

Next, we try to match each entry from the census data by county code. This is not ideal, as many counties fall into more than one congressional district, but it is the only way we can use this data at the congressional district level.

In [9]:
print('Census data column names')
print(df.columns.values)
print('df16 column names')
print(df16.columns.values)

Census data column names
['YEAR' 'DATANUM' 'SERIAL' 'HHWT' 'REGION' 'STATEICP' 'STATEFIP' 'COUNTY'
 'COUNTYFIPS' 'URBAN' 'METRO' 'CITYPOP' 'GQ' 'PERNUM' 'PERWT' 'SEX' 'AGE'
 'MARST' 'RACE' 'RACED' 'HISPAN' 'HISPAND' 'HCOVANY' 'HIGRADE' 'HIGRADED'
 'EDUC' 'EDUCD' 'EMPSTAT' 'EMPSTATD' 'OCC' 'IND' 'INCTOT']
df16 column names
['VTDID' 'PCTNAME' 'PCTCODE' 'MCDNAME' 'COUNTYNAME' 'COUNTYCODE'
 'CONGDIST' 'MNSENDIST' 'MNLEGDIST' 'CTYCOMDIST' 'JUDDIST' 'SWCDIST'
 'WARD' 'HOSPDIST' 'PARKDIST' 'TABSYSTEM' 'TABMODEL' 'MAILBALLOT' 'REG7AM'
 'EDR' 'SIGNATURES' 'AB_MB' 'FEDONLYAB' 'PRESONLYAB' 'TOTVOTING' 'USPRSR'
 'USPRSDFL' 'USPRSCP' 'USPRSLMN' 'USPRSSWP' 'USPRSGP' 'USPRSADP' 'USPRSIP'
 'USPRSLIB' 'USPRSWI' 'USPRSTOTAL' 'USREPR' 'USREPDFL' 'USREPWI'
 'USREPTOTAL' 'MNSENR' 'MNSENDFL' 'MNSENWI' 'MNSENTOTAL' 'MNLEGR'
 'MNLEGDFL' 'MNLEGWI' 'MNLEGTOTAL' 'MNCA1YES' 'MNCA1NO' 'MNCA1EST'
 'MNCA1TOTAL']


In [10]:
print(df16.COUNTYCODE.unique())
print(df.COUNTYFIPS.unique())
print('The number of empty County Fips code entries in df is: ',df.isna().sum().COUNTYFIPS)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71. 72.
 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. nan]
[ nan   0. 137.  53.  37. 163. 123. 109.   3. 145. 171.]
The number of empty County Fips code entries in df is:  458008


Unfortunately, about one fifth of our data is missing the county fips code and very few counties seem to have appeared. Let us see whether there is one county from each congressional district at least.

In [11]:
a=[0,137,53,37,163,123,109,3,145,171]
a.sort()
df16['COUNTYCODE']=pd.to_numeric(df16['COUNTYCODE'])
for i in a:
    temp=df16[df16['COUNTYCODE']==i]
    print('FIPS CODE %d'%(i),'unique count', temp.CONGDIST.unique())

FIPS CODE 0 unique count []
FIPS CODE 3 unique count [7.]
FIPS CODE 37 unique count [7.]
FIPS CODE 53 unique count [1.]
FIPS CODE 109 unique count []
FIPS CODE 123 unique count []
FIPS CODE 137 unique count []
FIPS CODE 145 unique count []
FIPS CODE 163 unique count []
FIPS CODE 171 unique count []


After matching these by hand (wikipedia list of counties) + state of secretary congressional district maps, it seems that we at least have one from each congressional district.

Next up, let us take a look at the general election and primary turnout data (state-wide).

In [12]:
# Let us start by loading the general turnout dataset.

turnout=[] #initialise a list
df_gen_primary=pd.read_csv("../Data sets/general_primary.csv")
df_gen_general=pd.read_csv("../Data sets/general_general.csv")

df_gen_primary.head() #Note that the first row contains the names of the columns.
df_gen_general.head()
# Rename columns, PercentEDV stands for percent voters registering on election day
# EDR stands for Election day registrations
    
df_gen_primary.columns=['Year','Eligible Voters Estimate','No of voters','Percent Turnout','EDR','PercentEDV']
df_gen_general.columns=['Year','Eligible Voters Estimate','No of voters','Percent Turnout','EDR','PercentEDV']

#DO THE BELOW ONCE - Removes first row and last row
df_gen_primary=df_gen_primary.drop([0,35]) #remove first and last row
df_gen_general=df_gen_general.drop([0,35])

#Remove '%' symbols
df_gen_general['Percent Turnout']=df_gen_general['Percent Turnout'].str.replace("%","")
df_gen_primary['Percent Turnout']=df_gen_primary['Percent Turnout'].str.replace("%","")

df_gen_general['Percent Turnout']=pd.to_numeric(df_gen_general['Percent Turnout'])
df_gen_primary['Percent Turnout']=pd.to_numeric(df_gen_primary['Percent Turnout'])


df_gen_general.head() #looks good. Let us move on
df_gen_primary.head()

,Year,Eligible Voters Estimate,No of voters,Percent Turnout,EDR,PercentEDV
1,1950,"1,879,000","583,617",31.06,No data,No data
2,1952,"1,899,000","652,825",34.38,No data,No data
3,1954,"1,920,000","641,906",33.43,No data,No data
4,1956,"1,940,000","611,197",31.51,No data,No data
5,1958,"1,960,000","632,240",32.26,No data,No data


### Master data set
Next, up we build a master data set that combines all of our data (to be used for machine learning). Some of the columns it'll include are
   - Year
   - CG (0 stands for the whole state)
   - Results (per party, per race etc)
   - Age, Income + other quantitative variables (percentages)
   - Sex, Race + other categorical variables (percentages)
   
#### <font color='red'> Notes: </font>
1. CG = 0 is for state-wide data
2. Unfortunately, each data set has different variable names, so we'll have to deal with each of them individually.
3. We won't use the years <2000 for the time being, since we are not sure we'll use them for training yet.

In [13]:
# Initialise year data frame
year=[]
for i in range(1994,2018,2):
    year=year+[i]*9

df_master=pd.DataFrame(columns=['YEAR','CG','CONGTOT','CONGIR','CONGDFL','CONGTH','SENTOT','SENIR','SENDFL','SENTH','GOVTOT',
                               'GOVIR','GOVDFL','GOVTH','AGE','INCOME'])
df_master['YEAR']=year
df_master['CG']=[0,1,2,3,4,5,6,7,8]*12

#### 2000 Results

In [14]:
#Fill this in for 2000

temp=df00.groupby('CG').sum() #group by congressional district

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[27+i,'CONGTH']=temp.loc[i,'IND_CONG']+temp.loc[i,'OTHER_CONG']+temp.loc[i,'CP_CONG']
    df_master.loc[27+i,'CONGIR']=temp.loc[i,'R_CONG']
    df_master.loc[27+i,'CONGDFL']=temp.loc[i,'DFL_CONG']
    df_master.loc[27+i,'CONGTOT']= df_master.loc[27+i,'CONGTH']+df_master.loc[27+i,'CONGIR']+ df_master.loc[27+i,'CONGDFL']
    # US Senator
    df_master.loc[27+i,'SENTH']=temp.loc[i,'IND_USSEN']+temp.loc[i,'OTHER_USSEN']+temp.loc[i,'CP_USSEN']+temp.loc[i,'SWP_USSEN']+temp.loc[i,'LIB_USSEN']+temp.loc[i,'GRP_USSEN']
    df_master.loc[27+i,'SENIR']=temp.loc[i,'R_USSEN']
    df_master.loc[27+i,'SENDFL']=temp.loc[i,'DFL_USSEN']
    df_master.loc[27+i,'SENTOT']= df_master.loc[27+i,'SENTH']+df_master.loc[27+i,'SENIR']+ df_master.loc[27+i,'SENDFL']

# Now tally-up to get the totals for the whole state

df_master.loc[27,:]=df_master.iloc[28:36,:].sum()
df_master.loc[27,'YEAR']=2000 #correct the year & CG
df_master.loc[27,'CG']=0

df_master.iloc[27:36,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
27,2000.0,0.0,2.38351e+06,993371,1.2342e+06,155931,2.46942e+06,1.04747e+06,1.18155e+06,240395,0,0,0,0,0,0
28,2000.0,1.0,283221,159835,117946,5440,292119,132346,135086,24687,NaN,NaN,NaN,NaN,NaN,NaN
29,2000.0,2.0,290799,139316,138969,12514,297993,148246,124316,25431,NaN,NaN,NaN,NaN,NaN,NaN
30,2000.0,3.0,330603,217933,101106,11564,345180,160770,150111,34299,NaN,NaN,NaN,NaN,NaN,NaN
31,2000.0,4.0,272724,83852,130403,58469,282725,96020,154687,32018,NaN,NaN,NaN,NaN,NaN,NaN
32,2000.0,5.0,256054,62470,173575,20009,277545,72124,165409,40012,NaN,NaN,NaN,NaN,NaN,NaN
33,2000.0,6.0,364408,170900,176340,17168,369641,170883,162743,36015,NaN,NaN,NaN,NaN,NaN,NaN
34,2000.0,7.0,276046,79175,185771,11100,282545,136151,124100,22294,NaN,NaN,NaN,NaN,NaN,NaN
35,2000.0,8.0,309651,79890,210094,19667,321674,130934,165101,25639,NaN,NaN,NaN,NaN,NaN,NaN


#### 2002 Results

In [15]:
# Next one. 2002
df02=pd.read_csv("C:\\Users\Michelle Pin\\Desktop\\Industry\\MinneMUDAC Data Science Challenge\\Data sets\\2002_results.csv")

df02.head()
# This one is somewhat of a mess, with no names for the columns.

#First let's remove the first two rows. DO THIS ONCE
df02=df02.drop([0,1])

df02.head()

,Precinct and Voting District Information,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,9TH DISTRICT COURT 4,Unnamed: 134,Unnamed: 135,9TH DISTRICT COURT 12,Unnamed: 137,Unnamed: 138,10TH DISTRICT COURT 14,Unnamed: 140,Unnamed: 141,Unnamed: 142
2,270010005,AITKIN,NaN,08,03B,01,00,001,00,00,...,368,336,3,479,202,3,0,0,0,NaN
3,270010010,AITKIN TWP.,NaN,08,03B,01,00,001,00,00,...,188,189,0,246,135,1,0,0,0,NaN
4,270010015,BALL BLUFF TWP.,NaN,08,03B,05,00,002,00,00,...,73,50,0,81,48,0,0,0,0,NaN
5,270010020,BALSAM TWP.,NaN,08,03B,05,00,004,00,00,...,5,2,0,4,6,0,0,0,0,NaN
6,270010025,BEAVER TWP.,NaN,08,03B,03,00,004,00,00,...,18,9,0,21,7,0,0,0,0,NaN


In [16]:
# Let us rename the columns
d=pd.read_csv("../Data sets/2002_cnames.csv")

cnames=['MCDPRCT']
for i in range(0,d.shape[0]):
    cnames.append(d.iloc[i,0])
cnames.append('mist') # an extra value at the end so that it matches the length of df02, not sure where this is coming from
df02.columns=cnames

In [17]:
# Convert columns to numeric
for i in ['CG','USSenGP','USSenIP','USSenR','USSenDFL','USSenCP','USSenWI','CongGP','CongIP','CongR','CongDFL','CongNNT','CongWI','GovGP','GovIP','GovR','GovDFL','GovCP','GovI','GovSW','GovWI']:
    df02[i]=pd.to_numeric(df02[i])
    
temp=df02.groupby('CG').sum() #group by congressional district
temp

,USSenGP,USSenIP,USSenR,USSenDFL,USSenCP,USSenWI,CongGP,CongIP,CongR,CongDFL,...,CongWI,GovGP,GovIP,GovR,GovDFL,GovCP,GovI,GovSW,GovWI,mist
CG,,,,,,,,,,,,,,,,,,,,,
1,1311,7819,132328,126543,302,300,9964,0,163570,92165,...,283,3842,95582,104596,63762,271,850,384,142,0.0
2,1039,5022,169460,114896,210,163,0,0,152970,121121,...,339,4842,48463,155040,82854,221,916,280,131,0.0
3,1041,4593,173412,123657,179,160,0,0,213334,82575,...,309,5033,43687,155825,97021,222,975,225,160,0.0
4,1258,3791,119848,148405,271,192,9919,0,89705,164597,...,319,8293,33231,110400,120863,360,1503,450,173,0.0
5,1948,4077,85660,169347,254,218,17825,0,66271,171572,...,314,13562,34607,77101,135782,258,1976,453,180,0.0
6,1206,5922,174089,111939,268,376,0,21484,164747,100738,...,343,5429,40023,159866,86493,352,1110,349,202,0.0
7,1008,7253,134746,122261,364,170,0,0,90342,170234,...,237,3802,33421,122268,103866,445,1140,461,159,0.0
8,1317,6688,127596,151100,408,225,0,0,88673,194909,...,349,5786,35520,114377,130627,408,1228,424,201,0.0


In [18]:
# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[36+i,'CONGTH']=temp.loc[i,'CongGP']+temp.loc[i,'CongIP']+temp.loc[i,'CongNNT']+temp.loc[i,'CongWI']
    df_master.loc[36+i,'CONGIR']=temp.loc[i,'CongR']
    df_master.loc[36+i,'CONGDFL']=temp.loc[i,'CongDFL']
    df_master.loc[36+i,'CONGTOT']= df_master.loc[36+i,'CONGTH']+df_master.loc[36+i,'CONGIR']+ df_master.loc[36+i,'CONGDFL']
    # US Senator
    df_master.loc[36+i,'SENTH']=temp.loc[i,'USSenGP']+temp.loc[i,'USSenIP']+temp.loc[i,'USSenCP']+temp.loc[i,'USSenWI']
    df_master.loc[36+i,'SENIR']=temp.loc[i,'USSenR']
    df_master.loc[36+i,'SENDFL']=temp.loc[i,'USSenDFL']
    df_master.loc[36+i,'SENTOT']= df_master.loc[36+i,'SENTH']+df_master.loc[36+i,'SENIR']+ df_master.loc[36+i,'SENDFL']

     # US Governor
    df_master.loc[36+i,'GOVTH']=temp.loc[i,'GovGP']+temp.loc[i,'GovIP']+temp.loc[i,'GovCP']+temp.loc[i,'GovI']+temp.loc[i,'GovSW']+temp.loc[i,'GovWI']
    df_master.loc[36+i,'GOVIR']=temp.loc[i,'GovR']
    df_master.loc[36+i,'GOVDFL']=temp.loc[i,'GovDFL']
    df_master.loc[36+i,'GOVTOT']= df_master.loc[36+i,'GOVTH']+df_master.loc[36+i,'GOVIR']+ df_master.loc[36+i,'GOVDFL']

    
# Now tally-up to get the totals for the whole state

df_master.loc[36,:]=df_master.iloc[37:45,:].sum()
df_master.loc[36,'YEAR']=2002 #correct the year & CG
df_master.loc[36,'CG']=0

# Check
df_master.iloc[36:45,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
36,2002.0,0.0,2.20164e+06,1.02961e+06,1.09791e+06,74115,2.24464e+06,1.11714e+06,1.06815e+06,59353,2.25247e+06,999473,821268,431732,0,0
37,2002.0,1.0,265982,163570,92165,10247,268603,132328,126543,9732,269429,104596,63762,101071,NaN,NaN
38,2002.0,2.0,286860,152970,121121,12769,290790,169460,114896,6434,292747,155040,82854,54853,NaN,NaN
39,2002.0,3.0,296218,213334,82575,309,303042,173412,123657,5973,303148,155825,97021,50302,NaN,NaN
40,2002.0,4.0,264540,89705,164597,10238,273765,119848,148405,5512,275273,110400,120863,44010,NaN,NaN
41,2002.0,5.0,255982,66271,171572,18139,261504,85660,169347,6497,263919,77101,135782,51036,NaN,NaN
42,2002.0,6.0,287312,164747,100738,21827,293800,174089,111939,7772,293824,159866,86493,47465,NaN,NaN
43,2002.0,7.0,260813,90342,170234,237,265802,134746,122261,8795,265562,122268,103866,39428,NaN,NaN
44,2002.0,8.0,283931,88673,194909,349,287334,127596,151100,8638,288571,114377,130627,43567,NaN,NaN


#### 2004 Results

In [19]:
# Next one. 2004

temp=df04.groupby('CG').sum() #group by congressional district

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[45+i,'CONGTH']=temp.loc[i,'USCongGr']+temp.loc[i,'USCongI']+temp.loc[i,'USCongWI']
    df_master.loc[45+i,'CONGIR']=temp.loc[i,'USCongR']
    df_master.loc[45+i,'CONGDFL']=temp.loc[i,'USCongDFL']
    df_master.loc[45+i,'CONGTOT']= df_master.loc[45+i,'CONGTH']+df_master.loc[45+i,'CONGIR']+ df_master.loc[45+i,'CONGDFL']
    
    '''
    # US Senator
    df_master.loc[36+i,'SENTH']=temp.loc[i,'USSenGP']+temp.loc[i,'USSenIP']+temp.loc[i,'USSenCP']+temp.loc[i,'USSenWI']
    df_master.loc[36+i,'SENIR']=temp.loc[i,'USSenR']
    df_master.loc[36+i,'SENDFL']=temp.loc[i,'USSenDFL']
    df_master.loc[36+i,'SENTOT']= df_master.loc[36+i,'SENTH']+df_master.loc[36+i,'SENIR']+ df_master.loc[36+i,'SENDFL']

     # US Governor
    df_master.loc[36+i,'GOVTH']=temp.loc[i,'GovGP']+temp.loc[i,'GovIP']+temp.loc[i,'GovCP']+temp.loc[i,'GovI']+temp.loc[i,'GovSW']+temp.loc[i,'GovWI']
    df_master.loc[36+i,'GOVIR']=temp.loc[i,'GovR']
    df_master.loc[36+i,'GOVDFL']=temp.loc[i,'GovDFL']
    df_master.loc[36+i,'GOVTOT']= df_master.loc[36+i,'GOVTH']+df_master.loc[36+i,'GOVIR']+ df_master.loc[36+i,'GOVDFL']
    '''
    
# Now tally-up to get the totals for the whole state

df_master.loc[45,:]=df_master.iloc[46:54,:].sum()
df_master.loc[45,'YEAR']=2004 #correct the year & CG
df_master.loc[45,'CG']=0

df_master[45:54]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
45,2004.0,0.0,2.72183e+06,1.2361e+06,1.39962e+06,86108,0,0,0,0,0,0,0,0,0,0
46,2004.0,1.0,324052,193133,115088,15831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2004.0,2.0,365945,206313,147527,12105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2004.0,3.0,358892,231871,126665,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2004.0,4.0,317299,105467,182387,29445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2004.0,5.0,313523,76600,218434,18489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,2004.0,6.0,377224,203669,173309,246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,2004.0,7.0,314257,106349,207628,280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,2004.0,8.0,350635,112693,228586,9356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2006 Results

In [20]:
# Next, 2006.

# Drop the last three rows because they contain gibberish. DO THIS ONCE
df06=df06.drop([4123,4124,4125])

# Convert columns to numeric
for i in ['CG','USSenR','USSenDFL','USSenIP','USSenWI','USSenTOT','GovR','GovDFL','GovIP','GovWI','GovTOT','CongR','CongDFL','CongIP','CongWI','CongTOT']: 
    df06[i]=pd.to_numeric(df06[i])
        
temp=df06.groupby('CG').sum() #group by congressional district

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[54+i,'CONGTH']=temp.loc[i,'CongIP']+temp.loc[i,'CongWI']
    df_master.loc[54+i,'CONGIR']=temp.loc[i,'CongR']
    df_master.loc[54+i,'CONGDFL']=temp.loc[i,'CongDFL']
    df_master.loc[54+i,'CONGTOT']= temp.loc[i,'CongTOT']
    
    # US Senator
    df_master.loc[54+i,'SENTH']=temp.loc[i,'USSenIP']+temp.loc[i,'USSenWI']
    df_master.loc[54+i,'SENIR']=temp.loc[i,'USSenR']
    df_master.loc[54+i,'SENDFL']=temp.loc[i,'USSenDFL']
    df_master.loc[54+i,'SENTOT']= temp.loc[i,'USSenTOT']

    # US Governor
    df_master.loc[54+i,'GOVTH']=temp.loc[i,'GovIP']+temp.loc[i,'GovWI']
    df_master.loc[54+i,'GOVIR']=temp.loc[i,'GovR']
    df_master.loc[54+i,'GOVDFL']=temp.loc[i,'GovDFL']
    df_master.loc[54+i,'GOVTOT']= temp.loc[i,'GovTOT']

    
#Now tally-up to get the totals for the whole state

df_master.loc[54,:]=df_master.iloc[55:63,:].sum()
df_master.loc[54,'YEAR']=2006 #correct the year & CG
df_master.loc[54,'CG']=0
df_master.iloc[54:63,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
54,2006.0,0.0,2.17897e+06,924636,1.15262e+06,88114,2.20277e+06,835653,1.27885e+06,72148,2.20294e+06,1.02857e+06,1.00746e+06,142684,0,0
55,2006.0,1.0,268421,126486,141556,379,268901,109372,147971,9482,268878,132466,117702,15036,NaN,NaN
56,2006.0,2.0,290540,163269,116343,10928,293453,125891,156714,9108,293755,160545,113052,17873,NaN,NaN
57,2006.0,3.0,284244,184333,99588,323,287975,118330,160041,8156,288024,151609,116074,18184,NaN,NaN
58,2006.0,4.0,247466,74797,172096,573,254322,76559,165913,9544,254588,96001,132250,23495,NaN,NaN
59,2006.0,5.0,244905,52263,136060,51790,249330,53006,184546,8875,248955,68121,155171,21946,NaN,NaN
60,2006.0,6.0,302188,151248,127144,23796,303334,138591,153247,9724,303371,169544,113109,18014,NaN,NaN
61,2006.0,7.0,257194,74557,179164,170,260046,110740,138849,8723,260063,132683,110809,13425,NaN,NaN
62,2006.0,8.0,284016,97683,180670,155,285411,103164,171568,8536,285303,117599,149293,14711,NaN,NaN


#### 2008 Results

In [21]:
temp=df08.groupby('CG').sum()

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[63+i,'CONGTH']=temp.loc[i,'CONGIP']+temp.loc[i,'CONGWI']
    df_master.loc[63+i,'CONGIR']=temp.loc[i,'CONGR']
    df_master.loc[63+i,'CONGDFL']=temp.loc[i,'CONGDFL']
    df_master.loc[63+i,'CONGTOT']= temp.loc[i,'CONGTOT']
    
    # US Senator
    df_master.loc[63+i,'SENTH']=temp.loc[i,'USSENIP']+temp.loc[i,'USSENLIB']+temp.loc[i,'USSENCP']+temp.loc[i,'USSENWI']
    df_master.loc[63+i,'SENIR']=temp.loc[i,'USSENR']
    df_master.loc[63+i,'SENDFL']=temp.loc[i,'USSENDFL']
    df_master.loc[63+i,'SENTOT']= temp.loc[i,'USSENTOT']
    
    '''
    # US Governor
    df_master.loc[63+i,'GOVTH']=temp.loc[i,'GovIP']+temp.loc[i,'GovWI']
    df_master.loc[63+i,'GOVIR']=temp.loc[i,'GovR']
    df_master.loc[63+i,'GOVDFL']=temp.loc[i,'GovDFL']
    df_master.loc[63+i,'GOVTOT']= temp.loc[i,'GovTOT']
    '''
    
#Now tally-up to get the totals for the whole state

df_master.loc[63,:]=df_master.iloc[63:72,:].sum()
df_master.loc[63,'YEAR']=2008 #correct the year & CG
df_master.loc[63,'CG']=0
df_master.iloc[63:72,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
63,2008.0,0.0,2.80261e+06,1.06902e+06,1.61248e+06,121119,2.88644e+06,1.2119e+06,1.21195e+06,462590,0,0,0,0,0,0
64,2008.0,1.0,332400,109453,207753,15194,337516,147745,132842,56929,NaN,NaN,NaN,NaN,NaN,NaN
65,2008.0,2.0,385656,220924,164093,639,396997,188408,140193,68396,NaN,NaN,NaN,NaN,NaN,NaN
66,2008.0,3.0,369104,178932,150787,39385,379212,176697,146915,55600,NaN,NaN,NaN,NaN,NaN,NaN
67,2008.0,4.0,316018,98936,216267,815,334982,117196,168445,49341,NaN,NaN,NaN,NaN,NaN,NaN
68,2008.0,5.0,322747,71020,228776,22951,339779,84817,209544,45418,NaN,NaN,NaN,NaN,NaN,NaN
69,2008.0,6.0,404725,187817,175786,41122,409630,199513,134917,75200,NaN,NaN,NaN,NaN,NaN,NaN
70,2008.0,7.0,314680,87062,227187,431,323010,150893,118020,54097,NaN,NaN,NaN,NaN,NaN,NaN
71,2008.0,8.0,357284,114871,241831,582,365315,146632,161074,57609,NaN,NaN,NaN,NaN,NaN,NaN


#### 2010 Results

In [22]:
temp=df10.groupby('CG').sum()

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[72+i,'CONGTH']=temp.loc[i,'CONGIP']+temp.loc[i,'CONGWI']
    df_master.loc[72+i,'CONGIR']=temp.loc[i,'CONGR']
    df_master.loc[72+i,'CONGDFL']=temp.loc[i,'CONGDFL']
    df_master.loc[72+i,'CONGTOT']= temp.loc[i,'CONGTOT']
    
    '''
    # US Senator
    df_master.loc[72+i,'SENTH']=temp.loc[i,'USSENIP']+temp.loc[i,'USSENLIB']+temp.loc[i,'USSENCP']+temp.loc[i,'USSENWI']
    df_master.loc[72+i,'SENIR']=temp.loc[i,'USSENR']
    df_master.loc[72+i,'SENDFL']=temp.loc[i,'USSENDFL']
    df_master.loc[72+i,'SENTOT']= temp.loc[i,'USSENTOT']
    
    '''
    # US Governor
    df_master.loc[72+i,'GOVTH']=temp.loc[i,'GOVIP']+temp.loc[i,'GOVGP']+temp.loc[i,'GOVWI']+temp.loc[i,'GOVTRP']+temp.loc[i,'GOVEDP']
    df_master.loc[72+i,'GOVIR']=temp.loc[i,'GOVR']
    df_master.loc[72+i,'GOVDFL']=temp.loc[i,'GOVDFL']
    df_master.loc[72+i,'GOVTOT']= temp.loc[i,'GOVTOT']

    
#Now tally-up to get the totals for the whole state

df_master.loc[72,:]=df_master.iloc[73:81,:].sum()
df_master.loc[72,'YEAR']=2010 #correct the year & CG
df_master.loc[72,'CG']=0
df_master.iloc[72:81,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
72,2010.0,0.0,2.09059e+06,970741,1.00203e+06,86455,0,0,0,0,2.10702e+06,910462,919232,269811,0,0
73,2010.0,1.0,248005,109242,122365,13344,NaN,NaN,NaN,NaN,247558,112742,98862,34667,NaN,NaN
74,2010.0,2.0,286453,181341,104809,303,NaN,NaN,NaN,NaN,289794,145660,103659,39699,NaN,NaN
75,2010.0,3.0,274092,161177,100240,12675,NaN,NaN,NaN,NaN,277624,131668,107190,38188,NaN,NaN
76,2010.0,4.0,231426,80141,136746,14539,NaN,NaN,NaN,NaN,235304,78957,124978,30694,NaN,NaN
77,2010.0,5.0,228746,55222,154833,7675,NaN,NaN,NaN,NaN,233192,54382,150691,27375,NaN,NaN
78,2010.0,6.0,303691,159476,120846,17879,NaN,NaN,NaN,NaN,304165,160619,105064,37451,NaN,NaN
79,2010.0,7.0,241097,90652,133096,8032,NaN,NaN,NaN,NaN,242258,114104,95167,31801,NaN,NaN
80,2010.0,8.0,277081,133490,129091,12008,NaN,NaN,NaN,NaN,277126,112330,133621,29936,NaN,NaN


#### 2012 Results

In [23]:
temp=df12.groupby('CONGDIST').sum()

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[81+i,'CONGTH']=temp.loc[i,'USREPIP']+temp.loc[i,'USREPWI']
    df_master.loc[81+i,'CONGIR']=temp.loc[i,'USREPR']
    df_master.loc[81+i,'CONGDFL']=temp.loc[i,'USREPDFL']
    df_master.loc[81+i,'CONGTOT']= temp.loc[i,'USREPTOTAL']
    
    
    # US Senator
    df_master.loc[81+i,'SENTH']=temp.loc[i,'USSENIP']+temp.loc[i,'USSENGR']+temp.loc[i,'USSENMOP']+temp.loc[i,'USSENWI']
    df_master.loc[81+i,'SENIR']=temp.loc[i,'USSENR']
    df_master.loc[81+i,'SENDFL']=temp.loc[i,'USSENDFL']
    df_master.loc[81+i,'SENTOT']= temp.loc[i,'USSENTOTAL']
    
    '''
    # US Governor
    df_master.loc[72+i,'GOVTH']=temp.loc[i,'GOVIP']+temp.loc[i,'GOVGP']+temp.loc[i,'GOVWI']+temp.loc[i,'GOVTRP']+temp.loc[i,'GOVEDP']
    df_master.loc[72+i,'GOVIR']=temp.loc[i,'GOVR']
    df_master.loc[72+i,'GOVDFL']=temp.loc[i,'GOVDFL']
    df_master.loc[72+i,'GOVTOT']= temp.loc[i,'GOVTOT']
    '''
    
#Now tally-up to get the totals for the whole state

df_master.loc[81,:]=df_master.iloc[81:90,:].sum()
df_master.loc[81,'YEAR']=2012 #correct the year & CG
df_master.loc[81,'CG']=0
df_master.iloc[81:90,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
81,2012.0,0.0,2.81338e+06,1.21041e+06,1.56098e+06,41990,2.84321e+06,867974,1.8546e+06,120638,0,0,0,0,0,0
82,2012.0,1.0,335880,142164,193211,505,332593,107055,207533,18005,NaN,NaN,NaN,NaN,NaN,NaN
83,2012.0,2.0,358446,193587,164338,521,366583,125988,226102,14493,NaN,NaN,NaN,NaN,NaN,NaN
84,2012.0,3.0,382705,222335,159937,433,391583,134333,244967,12283,NaN,NaN,NaN,NaN,NaN,NaN
85,2012.0,4.0,347991,109659,216685,21647,358571,88336,255214,15021,NaN,NaN,NaN,NaN,NaN,NaN
86,2012.0,5.0,351969,88753,262102,1114,361805,61738,284354,15713,NaN,NaN,NaN,NaN,NaN,NaN
87,2012.0,6.0,355153,179240,174944,969,354398,131551,207462,15385,NaN,NaN,NaN,NaN,NaN,NaN
88,2012.0,7.0,327576,114151,197791,15634,326883,111575,200024,15284,NaN,NaN,NaN,NaN,NaN,NaN
89,2012.0,8.0,353663,160520,191976,1167,350791,107398,228939,14454,NaN,NaN,NaN,NaN,NaN,NaN


#### 2014 Results

In [24]:
temp=df14.groupby('CONGDIST').sum()

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[90+i,'CONGTH']=temp.loc[i,'USREPIP']+temp.loc[i,'USREPWI']
    df_master.loc[90+i,'CONGIR']=temp.loc[i,'USREPR']
    df_master.loc[90+i,'CONGDFL']=temp.loc[i,'USREPDFL']
    df_master.loc[90+i,'CONGTOT']= temp.loc[i,'USREPTOTAL']
    
    
    # US Senator
    df_master.loc[90+i,'SENTH']=temp.loc[i,'USSENIP']+temp.loc[i,'USSENLIB']+temp.loc[i,'USSENWI']
    df_master.loc[90+i,'SENIR']=temp.loc[i,'USSENR']
    df_master.loc[90+i,'SENDFL']=temp.loc[i,'USSENDFL']
    df_master.loc[90+i,'SENTOT']= temp.loc[i,'USSENTOTAL']
    
    
    # US Governor
    df_master.loc[90+i,'GOVTH']=temp.loc[i,'MNGOVIP']+temp.loc[i,'MNGOVLIB']+temp.loc[i,'MNGOVLIB']+temp.loc[i,'MNGOVGLC']+temp.loc[i,'MNGOVWI']
    df_master.loc[90+i,'GOVIR']=temp.loc[i,'MNGOVR']
    df_master.loc[90+i,'GOVDFL']=temp.loc[i,'MNGOVDFL']
    df_master.loc[90+i,'GOVTOT']= temp.loc[i,'MNGOVTOTAL']
    
    
#Now tally-up to get the totals for the whole state

df_master.loc[90,:]=df_master.iloc[91:99,:].sum()
df_master.loc[90,'YEAR']=2014 #correct the year & CG
df_master.loc[90,'CG']=0
df_master.iloc[90:99,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
90,2014.0,0.0,1.96354e+06,913539,985760,52790,1.98153e+06,850227,1.05320e+06,78096,1.97541e+06,879257,989113,125118,0,0
91,2014.0,1.0,226695,103536,122851,308,227896,103135,112651,12110,227062,109803,103235,16070,NaN,NaN
92,2014.0,2.0,245848,137778,95565,12505,247680,118073,120679,8928,247075,121203,113476,14845,NaN,NaN
93,2014.0,3.0,269585,167515,101846,224,272337,132492,132176,7669,271967,135183,125866,13182,NaN,NaN
94,2014.0,4.0,241637,79492,147857,14288,245159,86427,150673,8059,244309,88571,143302,14659,NaN,NaN
95,2014.0,5.0,236010,56577,167079,12354,239390,54666,177016,7708,238855,55275,169222,16950,NaN,NaN
96,2014.0,6.0,236846,133328,90926,12592,239354,128359,101234,9761,238608,132298,93140,15304,NaN,NaN
97,2014.0,7.0,240835,109955,130546,334,242617,114717,115506,12394,241519,121717,105359,16459,NaN,NaN
98,2014.0,8.0,266083,125358,129090,185,267095,112358,143270,11467,266011,115207,135513,17649,NaN,NaN


#### 2016 Results

In [25]:
temp=df16.groupby('CONGDIST').sum()

# Now let's do this for each congressional district
for i in range(1,9):
    # Congressional district house
    df_master.loc[99+i,'CONGTH']=temp.loc[i,'USREPWI']
    df_master.loc[99+i,'CONGIR']=temp.loc[i,'USREPR']
    df_master.loc[99+i,'CONGDFL']=temp.loc[i,'USREPDFL']
    df_master.loc[99+i,'CONGTOT']= temp.loc[i,'USREPTOTAL']
    
    '''
    
    # US Senator
    df_master.loc[99+i,'SENTH']=temp.loc[i,'USSENIP']+temp.loc[i,'USSENLIB']+temp.loc[i,'USSENWI']
    df_master.loc[99+i,'SENIR']=temp.loc[i,'USSENR']
    df_master.loc[99+i,'SENDFL']=temp.loc[i,'USSENDFL']
    df_master.loc[99+i,'SENTOT']= temp.loc[i,'USSENTOTAL']
    
    
    # US Governor
    df_master.loc[99+i,'GOVTH']=temp.loc[i,'MNGOVIP']+temp.loc[i,'MNGOVLIB']+temp.loc[i,'MNGOVLIB']+temp.loc[i,'MNGOVGLC']+temp.loc[i,'MNGOVWI']
    df_master.loc[99+i,'GOVIR']=temp.loc[i,'MNGOVR']
    df_master.loc[99+i,'GOVDFL']=temp.loc[i,'MNGOVDFL']
    df_master.loc[99+i,'GOVTOT']= temp.loc[i,'MNGOVTOTAL']
    '''
    
#Now tally-up to get the totals for the whole state

df_master.loc[99,:]=df_master.iloc[100:108,:].sum()
df_master.loc[99,'YEAR']=2016 #correct the year & CG
df_master.loc[99,'CG']=0
df_master.iloc[99:108,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,GOVTOT,GOVIR,GOVDFL,GOVTH,AGE,INCOME
99,2016.0,0.0,2.86043e+06,1.33469e+06,1.43459e+06,4376,0,0,0,0,0,0,0,0,0,0
100,2016.0,1.0,335877,166526,169074,277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,2016.0,2.0,370514,173970,167315,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,2016.0,3.0,393464,223077,169243,1144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,2016.0,4.0,351944,121032,203299,461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,2016.0,5.0,361882,80660,249964,499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2016.0,6.0,358924,235380,123008,536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,2016.0,7.0,330848,156952,173589,307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,2016.0,8.0,356979,177089,179098,792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Let us save our progress so far.

df_master.to_csv('df_master_1.csv')

Next, let us take a look at census data. (Sarah added data to the master set from above)

In [27]:
df_master=pd.read_csv('../Data sets/df_master3.csv')
df_master=df_master.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
df_master.head()

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,SEX_1_RACE_9,SEX_2_RACE_1,SEX_2_RACE_2,SEX_2_RACE_4,SEX_2_RACE_3,SEX_2_RACE_8,SEX_2_RACE_6,SEX_2_RACE_7,SEX_2_RACE_5,SEX_2_RACE_9
0,1994,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1994,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1994,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1994,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1994,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
list(df_master.columns.values)

['YEAR',
 'CG',
 'CONGTOT',
 'CONGIR',
 'CONGDFL',
 'CONGTH',
 'SENTOT',
 'SENIR',
 'SENDFL',
 'SENTH',
 'GOVTOT',
 'GOVIR',
 'GOVDFL',
 'GOVTH',
 'AGE0-18',
 'AGE19-36',
 'AGE37-52',
 'AGE53-70',
 'AGE71-100',
 'INC0-10k',
 'INC10k-20k',
 'INC20k-40k',
 'INC40k-60k',
 'INC60k-100k',
 'INC>100k',
 'EDUC_0.0',
 'EDUC_1.0',
 'EDUC_10.0',
 'EDUC_11.0',
 'EDUC_2.0',
 'EDUC_3.0',
 'EDUC_4.0',
 'EDUC_5.0',
 'EDUC_6.0',
 'EDUC_7.0',
 'EDUC_8.0',
 'EMPSTAT_0.0',
 'EMPSTAT_1.0',
 'EMPSTAT_2.0',
 'EMPSTAT_3.0',
 'HCOVANY_1.0',
 'HCOVANY_2.0',
 'HISPAN_0',
 'HISPAN_1',
 'HISPAN_2',
 'HISPAN_3',
 'HISPAN_4',
 'IND_0.0',
 'IND_10.0',
 'IND_100.0',
 'IND_101.0',
 'IND_102.0',
 'IND_107.0',
 'IND_1070.0',
 'IND_108.0',
 'IND_1080.0',
 'IND_109.0',
 'IND_1090.0',
 'IND_11.0',
 'IND_110.0',
 'IND_111.0',
 'IND_112.0',
 'IND_117.0',
 'IND_1170.0',
 'IND_118.0',
 'IND_1180.0',
 'IND_119.0',
 'IND_1190.0',
 'IND_12.0',
 'IND_120.0',
 'IND_121.0',
 'IND_122.0',
 'IND_127.0',
 'IND_1270.0',
 'IND_128.0',
 '

In [29]:
# There are way to many ind variables so let us throw those out
temp=[]
for d in range(0,10000):
    temp.append('IND_%d.0'%(d))
# Not all of these are in the columns names so let us pick the ones that are
temp2=[]
for d in temp:
    if d in df_master.columns.values:
        temp2.append(d)
# Get rid of these columns then
df_master=df_master.drop(columns=temp2)
list(df_master.columns.values)

['YEAR',
 'CG',
 'CONGTOT',
 'CONGIR',
 'CONGDFL',
 'CONGTH',
 'SENTOT',
 'SENIR',
 'SENDFL',
 'SENTH',
 'GOVTOT',
 'GOVIR',
 'GOVDFL',
 'GOVTH',
 'AGE0-18',
 'AGE19-36',
 'AGE37-52',
 'AGE53-70',
 'AGE71-100',
 'INC0-10k',
 'INC10k-20k',
 'INC20k-40k',
 'INC40k-60k',
 'INC60k-100k',
 'INC>100k',
 'EDUC_0.0',
 'EDUC_1.0',
 'EDUC_10.0',
 'EDUC_11.0',
 'EDUC_2.0',
 'EDUC_3.0',
 'EDUC_4.0',
 'EDUC_5.0',
 'EDUC_6.0',
 'EDUC_7.0',
 'EDUC_8.0',
 'EMPSTAT_0.0',
 'EMPSTAT_1.0',
 'EMPSTAT_2.0',
 'EMPSTAT_3.0',
 'HCOVANY_1.0',
 'HCOVANY_2.0',
 'HISPAN_0',
 'HISPAN_1',
 'HISPAN_2',
 'HISPAN_3',
 'HISPAN_4',
 'MARST_1.0',
 'MARST_2.0',
 'MARST_3.0',
 'MARST_4.0',
 'MARST_5.0',
 'MARST_6.0',
 'METRO_0.0',
 'METRO_1.0',
 'METRO_2.0',
 'METRO_3.0',
 'METRO_4.0',
 'RACE_1',
 'RACE_2',
 'RACE_3',
 'RACE_4',
 'RACE_5',
 'RACE_6',
 'RACE_7',
 'RACE_8',
 'RACE_9',
 'SEX_1',
 'SEX_2',
 'URBAN_1.0',
 'URBAN_2.0',
 'URBAN_nan',
 'METRO_nan',
 'MARST_nan',
 'HCOVANY_nan',
 'EDUC_nan',
 'EDUC_9.0',
 'EMPSTAT_n

Next, we add
- state-wide turnout data for general elections and primaries 
- weather data (state-wide)

In [30]:
df_gen=pd.read_csv('../Data Sets/general_general.csv')
df_pr=pd.read_csv('../Data Sets/general_primary.csv')
# Let us fix the column names and delete the first row and last
df_gen.columns=['Year', 'Eligible_voters','Number_voters','Per_turnout','EDR','PerEDR']
df_pr.columns=['Year', 'Eligible_voters','Number_voters','Per_turnout','EDR','PerEDR']
df_gen= df_gen.drop([0,35])
df_pr=df_pr.drop([0,35])
df_gen.head()

,Year,Eligible_voters,Number_voters,Per_turnout,EDR,PerEDR
1,1950,"1,879,000","1,067,967",56.84%,No data,No data
2,1952,"1,899,000","1,466,326",77.22%,No data,No data
3,1954,"1,920,000","1,168,101",60.84%,No data,No data
4,1956,"1,940,000","1,613,138",83.15%,No data,No data
5,1958,"1,960,000","1,178,173",60.11%,No data,No data


In [31]:
# Looking good. Now let's add this to the master data set.

df_gen['Year']=pd.to_numeric(df_gen['Year'])
df_pr['Year']=pd.to_numeric(df_pr['Year'])

#Initialise lists to hold percent turnout (pt) and eligible voters (el)
prim_pt=[]
prim_el=[]
gen_pt=[]
gen_el=[]

for y in range(2000,2018,2):
    prim_pt.append(df_pr[df_pr['Year']==y].iloc[0]['Per_turnout'])
    prim_el.append(df_pr[df_pr['Year']==y].iloc[0]['Eligible_voters'])
    gen_pt.append(df_gen[df_gen['Year']==y].iloc[0]['Per_turnout'])
    gen_el.append(df_gen[df_gen['Year']==y].iloc[0]['Eligible_voters'])
    
# Now let us remove the percent, commas and convert to numeric
prim_pt = [float(item.rstrip('%')) for item in prim_pt]
prim_el = [int(item.replace(',','')) for item in prim_el]
gen_pt = [float(item.rstrip('%')) for item in gen_pt]
gem_el = [int(item.replace(',','')) for item in gen_el]

# Now let us add this to our data set

# Initialise new columns
df_master['prim_pt']=np.full(df_master.shape[0],np.nan)
df_master['prim_el']=np.full(df_master.shape[0],np.nan)
df_master['gen_pt']=np.full(df_master.shape[0],np.nan)
df_master['gen_el']=np.full(df_master.shape[0],np.nan)

# find column indeces
# print(df_master.columns.get_loc('prim_pt'))

for i in range(27,108,9):
    df_master.iloc[i,97]=prim_pt[int(i/9-3)]
    df_master.iloc[i,98]=prim_el[int(i/9-3)]
    df_master.iloc[i,99]=gen_pt[int(i/9-3)]
    df_master.iloc[i,100]=gen_el[int(i/9-3)]
    
# Quick check
df_master.iloc[range(27,108,9),97:101]

,prim_pt,prim_el,gen_pt,gen_el
27,17.19,3506432.0,70.11,"3,506,432"
36,15.23,3518184.0,64.89,"3,518,184"
45,7.73,3609185.0,78.77,"3,609,185"
54,14.02,3667707.0,60.47,"3,667,707"
63,11.22,3740142.0,78.11,"3,740,142"
72,15.94,3803192.0,55.83,"3,803,192"
81,9.32,3861043.0,76.42,"3,861,043"
90,10.19,3945136.0,50.51,"3,945,136"
99,7.42,3972330.0,74.72,"3,972,330"


#### State-wide level data

In [32]:
# Looks good. Now let us concentrate on state-wide level data
df_master['CG']=pd.to_numeric(df_master['CG'])
df=df_master.iloc[range(27,108,9),:]

### Let us add weather data on election day (MSP station)
df['high']=[48,36,48,61,71,56,44,53,59]
df['low']=[30,30,36,44,51,34,38,37,39]
df['Prec']=[0.06,0.03,0.06,0,0,0,0.31,0,0]
df['wind']=[24,12,17,15,25,17,20,29,17]

#Let us save this state-wide level data
df.to_csv('df_state_2000.csv')

#### Percentages

Using the general turnout numbers, we compute the percentages of turnout for each congressional district.

#### <font color='red'> Important Note </font>
The general turnout differs from CONGTOT (probably because many people just vote for president). This difference is recorded in a variable called 'dif_tot'.

In [33]:
df_master.columns.values

array(['YEAR', 'CG', 'CONGTOT', 'CONGIR', 'CONGDFL', 'CONGTH', 'SENTOT',
       'SENIR', 'SENDFL', 'SENTH', 'GOVTOT', 'GOVIR', 'GOVDFL', 'GOVTH',
       'AGE0-18', 'AGE19-36', 'AGE37-52', 'AGE53-70', 'AGE71-100',
       'INC0-10k', 'INC10k-20k', 'INC20k-40k', 'INC40k-60k',
       'INC60k-100k', 'INC>100k', 'EDUC_0.0', 'EDUC_1.0', 'EDUC_10.0',
       'EDUC_11.0', 'EDUC_2.0', 'EDUC_3.0', 'EDUC_4.0', 'EDUC_5.0',
       'EDUC_6.0', 'EDUC_7.0', 'EDUC_8.0', 'EMPSTAT_0.0', 'EMPSTAT_1.0',
       'EMPSTAT_2.0', 'EMPSTAT_3.0', 'HCOVANY_1.0', 'HCOVANY_2.0',
       'HISPAN_0', 'HISPAN_1', 'HISPAN_2', 'HISPAN_3', 'HISPAN_4',
       'MARST_1.0', 'MARST_2.0', 'MARST_3.0', 'MARST_4.0', 'MARST_5.0',
       'MARST_6.0', 'METRO_0.0', 'METRO_1.0', 'METRO_2.0', 'METRO_3.0',
       'METRO_4.0', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4', 'RACE_5',
       'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9', 'SEX_1', 'SEX_2',
       'URBAN_1.0', 'URBAN_2.0', 'URBAN_nan', 'METRO_nan', 'MARST_nan',
       'HCOVANY_nan', 'EDUC_n

In [34]:
# First, let us add some empty columns to hold our new variables.
df_master['congtot_pt']=np.full(df_master.shape[0],np.nan)
df_master['congir_pt']=np.full(df_master.shape[0],np.nan)
df_master['congdfl_pt']=np.full(df_master.shape[0],np.nan)
df_master['sentot_pt']=np.full(df_master.shape[0],np.nan)
df_master['senir_pt']=np.full(df_master.shape[0],np.nan)
df_master['sendfl_pt']=np.full(df_master.shape[0],np.nan)
df_master['govtot_pt']=np.full(df_master.shape[0],np.nan)
df_master['govir_pt']=np.full(df_master.shape[0],np.nan)
df_master['govdfl_pt']=np.full(df_master.shape[0],np.nan)
df_master['dif_tot']=np.full(df_master.shape[0],np.nan)

# Strip gen_el field from commas
df_master['gen_el']=df_master['gen_el'].str.replace(',','')
# Convert to integer
df_master['gen_el']=pd.to_numeric(df_master['gen_el'])
df_master['gen_pt']=pd.to_numeric(df_master['gen_pt'])

# Now compute percent turnout by congressional district for the years 2000-20016
for i in range(27,108,9):
    temp=df_master.iloc[i,:]
    df_master.loc[i,'dif_tot']=temp['gen_pt']/100*temp['gen_el']-temp['CONGTOT']
    total=temp['CONGTOT'] #Total number of voters that turned up
    df_master.loc[i:i+8,'congtot_pt']=df_master.loc[i:i+8,'CONGTOT']/total
    df_master.loc[i:i+8,'congir_pt']=np.divide(df_master.loc[i:i+8,'CONGIR'],df_master.loc[i:i+8,'CONGTOT'])
    df_master.loc[i:i+8,'congdfl_pt']=np.divide(df_master.loc[i:i+8,'CONGDFL'],df_master.loc[i:i+8,'CONGTOT'])
    df_master.loc[i:i+8,'sentot_pt']=df_master.loc[i:i+8,'SENTOT']/total
    df_master.loc[i:i+8,'senir_pt']=np.divide(df_master.loc[i:i+8,'SENIR'],df_master.loc[i:i+8,'SENTOT'])
    df_master.loc[i:i+8,'sendfl_pt']=np.divide(df_master.loc[i:i+8,'SENDFL'],df_master.loc[i:i+8,'SENTOT'])
    df_master.loc[i:i+8,'govtot_pt']=df_master.loc[i:i+8,'GOVTOT']/total
    df_master.loc[i:i+8,'govir_pt']=np.divide(df_master.loc[i:i+8,'GOVIR'],df_master.loc[i:i+8,'GOVTOT'])
    df_master.loc[i:i+8,'govdfl_pt']=np.divide(df_master.loc[i:i+8,'GOVDFL'],df_master.loc[i:i+8,'GOVTOT'])
    
# Save this

df_master.to_csv('df_master_percent.csv')

### Merge with corrected census data
Next, we need to merge our data set with the new census data (there was a computation error in the first set I was given, so now i need to add the corrected data).

In [95]:
df_master=pd.read_csv('df_master_percent.csv')
df_census=pd.read_csv('../Data Sets/clean_census_data.csv')

In [79]:
df_census.columns.values

array(['Unnamed: 0', 'AGE0-17', 'AGE18-34', 'AGE35-49', 'AGE50-69',
       'AGE70-100', 'EDUC_0.0', 'EDUC_1.0', 'EDUC_10.0', 'EDUC_11.0',
       'EDUC_2.0', 'EDUC_3.0', 'EDUC_4.0', 'EDUC_5.0', 'EDUC_6.0',
       'EDUC_7.0', 'EDUC_8.0', 'EDUC_9.0', 'EMPSTAT_0.0', 'EMPSTAT_1.0',
       'EMPSTAT_2.0', 'EMPSTAT_3.0', 'HCOVANY_1.0', 'HCOVANY_2.0',
       'HISPAN_0', 'HISPAN_1', 'HISPAN_2', 'HISPAN_3', 'HISPAN_4',
       'INC0-10k', 'INC10k-20k', 'INC20k-40k', 'INC40k-60k',
       'INC60k-100k', 'INC>100k', 'MARST_1.0', 'MARST_2.0', 'MARST_3.0',
       'MARST_4.0', 'MARST_5.0', 'MARST_6.0', 'METRO_0.0', 'METRO_1.0',
       'METRO_2.0', 'METRO_3.0', 'METRO_4.0', 'RACE_1', 'RACE_2',
       'RACE_3', 'RACE_4', 'RACE_5', 'RACE_6', 'RACE_7', 'RACE_8',
       'RACE_9', 'SEX_1', 'SEX_2', 'URBAN_1.0', 'URBAN_2.0', 'YEAR', 'CG'],
      dtype=object)

In [41]:
df_master.columns.values

array(['Unnamed: 0', 'YEAR', 'CG', 'CONGTOT', 'CONGIR', 'CONGDFL',
       'CONGTH', 'SENTOT', 'SENIR', 'SENDFL', 'SENTH', 'GOVTOT', 'GOVIR',
       'GOVDFL', 'GOVTH', 'AGE0-18', 'AGE19-36', 'AGE37-52', 'AGE53-70',
       'AGE71-100', 'INC0-10k', 'INC10k-20k', 'INC20k-40k', 'INC40k-60k',
       'INC60k-100k', 'INC>100k', 'EDUC_0.0', 'EDUC_1.0', 'EDUC_10.0',
       'EDUC_11.0', 'EDUC_2.0', 'EDUC_3.0', 'EDUC_4.0', 'EDUC_5.0',
       'EDUC_6.0', 'EDUC_7.0', 'EDUC_8.0', 'EMPSTAT_0.0', 'EMPSTAT_1.0',
       'EMPSTAT_2.0', 'EMPSTAT_3.0', 'HCOVANY_1.0', 'HCOVANY_2.0',
       'HISPAN_0', 'HISPAN_1', 'HISPAN_2', 'HISPAN_3', 'HISPAN_4',
       'MARST_1.0', 'MARST_2.0', 'MARST_3.0', 'MARST_4.0', 'MARST_5.0',
       'MARST_6.0', 'METRO_0.0', 'METRO_1.0', 'METRO_2.0', 'METRO_3.0',
       'METRO_4.0', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4', 'RACE_5',
       'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9', 'SEX_1', 'SEX_2',
       'URBAN_1.0', 'URBAN_2.0', 'URBAN_nan', 'METRO_nan', 'MARST_nan',
       'HCOVANY

In [96]:
# We want to drop some columns we won't be using. Let us first find their column indeces.

print(df_master.columns.get_loc('URBAN_nan'))
print(df_master.columns.get_loc('SEX_2_RACE_9'))

72
97


In [97]:
df_census=df_census.drop(columns=['EDUC_9.0','Unnamed: 0'])
df_master=df_master.drop(df_master.columns[72:98], axis=1)
df_master=df_master.drop(columns=['Unnamed: 0'])
df_master.columns.values

array(['YEAR', 'CG', 'CONGTOT', 'CONGIR', 'CONGDFL', 'CONGTH', 'SENTOT',
       'SENIR', 'SENDFL', 'SENTH', 'GOVTOT', 'GOVIR', 'GOVDFL', 'GOVTH',
       'AGE0-18', 'AGE19-36', 'AGE37-52', 'AGE53-70', 'AGE71-100',
       'INC0-10k', 'INC10k-20k', 'INC20k-40k', 'INC40k-60k',
       'INC60k-100k', 'INC>100k', 'EDUC_0.0', 'EDUC_1.0', 'EDUC_10.0',
       'EDUC_11.0', 'EDUC_2.0', 'EDUC_3.0', 'EDUC_4.0', 'EDUC_5.0',
       'EDUC_6.0', 'EDUC_7.0', 'EDUC_8.0', 'EMPSTAT_0.0', 'EMPSTAT_1.0',
       'EMPSTAT_2.0', 'EMPSTAT_3.0', 'HCOVANY_1.0', 'HCOVANY_2.0',
       'HISPAN_0', 'HISPAN_1', 'HISPAN_2', 'HISPAN_3', 'HISPAN_4',
       'MARST_1.0', 'MARST_2.0', 'MARST_3.0', 'MARST_4.0', 'MARST_5.0',
       'MARST_6.0', 'METRO_0.0', 'METRO_1.0', 'METRO_2.0', 'METRO_3.0',
       'METRO_4.0', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4', 'RACE_5',
       'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9', 'SEX_1', 'SEX_2',
       'URBAN_1.0', 'URBAN_2.0', 'prim_pt', 'prim_el', 'gen_pt', 'gen_el',
       'congtot_pt', 'cong

In [98]:
# New age bins were used for the census data, so we need to rename the age columns in df_master.
print(df_census.columns.get_loc('AGE0-17'))
print(df_census.columns.get_loc('AGE70-100'))
df_master.rename(columns={'AGE0-18':'AGE0-17', 'AGE19-36':'AGE18-34', 'AGE37-52':'AGE35-49', 'AGE53-70':'AGE50-69','AGE71-100':'AGE70-100'}, inplace=True)
df_master.columns.values

0
4


array(['YEAR', 'CG', 'CONGTOT', 'CONGIR', 'CONGDFL', 'CONGTH', 'SENTOT',
       'SENIR', 'SENDFL', 'SENTH', 'GOVTOT', 'GOVIR', 'GOVDFL', 'GOVTH',
       'AGE0-17', 'AGE18-34', 'AGE35-49', 'AGE50-69', 'AGE70-100',
       'INC0-10k', 'INC10k-20k', 'INC20k-40k', 'INC40k-60k',
       'INC60k-100k', 'INC>100k', 'EDUC_0.0', 'EDUC_1.0', 'EDUC_10.0',
       'EDUC_11.0', 'EDUC_2.0', 'EDUC_3.0', 'EDUC_4.0', 'EDUC_5.0',
       'EDUC_6.0', 'EDUC_7.0', 'EDUC_8.0', 'EMPSTAT_0.0', 'EMPSTAT_1.0',
       'EMPSTAT_2.0', 'EMPSTAT_3.0', 'HCOVANY_1.0', 'HCOVANY_2.0',
       'HISPAN_0', 'HISPAN_1', 'HISPAN_2', 'HISPAN_3', 'HISPAN_4',
       'MARST_1.0', 'MARST_2.0', 'MARST_3.0', 'MARST_4.0', 'MARST_5.0',
       'MARST_6.0', 'METRO_0.0', 'METRO_1.0', 'METRO_2.0', 'METRO_3.0',
       'METRO_4.0', 'RACE_1', 'RACE_2', 'RACE_3', 'RACE_4', 'RACE_5',
       'RACE_6', 'RACE_7', 'RACE_8', 'RACE_9', 'SEX_1', 'SEX_2',
       'URBAN_1.0', 'URBAN_2.0', 'prim_pt', 'prim_el', 'gen_pt', 'gen_el',
       'congtot_pt', 'cong

In [99]:
# Quick reordering of columns in census data

print(df_master.columns.get_loc('AGE0-17'))
print(df_master.columns.get_loc('URBAN_2.0'))

temp=['YEAR','CG']
for i in range(14,71):
    temp.append(df_master.columns[i])
df_cen=df_census[temp] # put columns in the right order
df_cen.head()

14
70


,YEAR,CG,AGE0-17,AGE18-34,AGE35-49,AGE50-69,AGE70-100,INC0-10k,INC10k-20k,INC20k-40k,...,RACE_4,RACE_5,RACE_6,RACE_7,RACE_8,RACE_9,SEX_1,SEX_2,URBAN_1.0,URBAN_2.0
0,1850,0.0,0.481481,0.314815,0.185185,0.018519,0.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.0,NaN,NaN,0.517241,0.482759,1.000000,0.000000
1,1860,0.0,0.460327,0.324652,0.153240,0.052695,0.009085,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.0,NaN,NaN,0.554513,0.445487,0.877491,0.122509
2,1870,0.0,0.468289,0.271773,0.168379,0.080840,0.010719,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.0,NaN,NaN,0.527952,0.472048,0.838118,0.161882
3,1880,0.0,0.435294,0.305503,0.150366,0.095477,0.013359,NaN,NaN,NaN,...,0.000114,0.000000,0.0,0.0,NaN,NaN,0.536809,0.463191,0.807428,0.192572
4,1900,0.0,0.405842,0.300195,0.173483,0.098748,0.021733,NaN,NaN,NaN,...,0.000112,0.000078,0.0,0.0,NaN,NaN,0.532982,0.467018,0.657736,0.342264


Next, we go through df_master and add the relevant data when it exists. Because of the nature of the data, we might have to do that for each 'chunk of years separately.

#### 1990

In [100]:
df_cen['YEAR']=pd.to_numeric(df_cen['YEAR'])
df_cen['CG']=pd.to_numeric(df_cen['CG'])

# First let us do 1990. Let us add another 8 rows for 1990
s2 = pd.Series(np.full(df_master.shape[1],np.nan), index=df_master.columns) #empty row

for i in range(0,9):
    df_master=df_master.append(s2,ignore_index=True)
df_master.shape
df_cen['YEAR']=pd.to_numeric(df_cen['YEAR'])
df_cen['CG']=pd.to_numeric(df_cen['CG'])

block=df_cen[df_cen['YEAR']==1990]

temp.remove('YEAR') #We don't need these anymore
temp.remove('CG')

for i in [0,1,2,3,4,5,7,8]:
    row=block[block['CG']==i]
    df_master.loc[108+i,'YEAR']=1990
    df_master.loc[108+i,'CG']=i
    for j in temp:
        df_master.loc[108+i,j]=row.iloc[0][j]
        
df_master.iloc[108:117,20:40]

,INC10k-20k,INC20k-40k,INC40k-60k,INC60k-100k,INC>100k,EDUC_0.0,EDUC_1.0,EDUC_10.0,EDUC_11.0,EDUC_2.0,EDUC_3.0,EDUC_4.0,EDUC_5.0,EDUC_6.0,EDUC_7.0,EDUC_8.0,EMPSTAT_0.0,EMPSTAT_1.0,EMPSTAT_2.0,EMPSTAT_3.0
108,0.275503,0.226145,0.040482,0.014589,0.004972,0.085839,0.108073,0.061488,0.020372,0.149869,0.031939,0.036357,0.033036,0.287528,0.125154,0.060345,0.262329,0.453941,0.027551,0.256179
109,0.251994,0.290271,0.086762,0.030622,0.018820,0.088671,0.113508,0.111547,0.056209,0.094336,0.025926,0.028322,0.025490,0.233769,0.141830,0.080392,0.266885,0.531155,0.016993,0.184967
110,0.225156,0.326719,0.103438,0.040938,0.014375,0.086545,0.112306,0.131467,0.036938,0.087928,0.022887,0.026080,0.027358,0.245263,0.158293,0.064935,0.267831,0.559293,0.019055,0.153822
111,0.256335,0.336321,0.084641,0.019824,0.006723,0.096383,0.106884,0.072579,0.021820,0.085648,0.026021,0.034072,0.033139,0.291365,0.165928,0.066161,0.271295,0.548191,0.026021,0.154492
112,0.247357,0.290879,0.087728,0.032744,0.013985,0.091369,0.095998,0.135246,0.059257,0.081196,0.020733,0.029171,0.026760,0.238910,0.165477,0.055882,0.240839,0.521504,0.025265,0.212392
113,0.253444,0.289766,0.085855,0.037049,0.022949,0.079930,0.089783,0.160068,0.059032,0.067992,0.020928,0.026435,0.027060,0.222136,0.183824,0.062813,0.214307,0.548107,0.026107,0.211479
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,0.281330,0.235193,0.046567,0.015021,0.008369,0.089161,0.112845,0.060463,0.027584,0.159376,0.029395,0.029395,0.024937,0.279186,0.131792,0.055865,0.281973,0.483979,0.025495,0.208554
116,0.241430,0.249745,0.045952,0.010941,0.004376,0.072674,0.098881,0.076504,0.031146,0.102611,0.033868,0.039210,0.038907,0.281927,0.160568,0.063703,0.233847,0.418607,0.037194,0.310352


#### Now add the years before 1990


In [101]:
block=df_cen[df_cen['YEAR']<1990]
print(block.shape) # So we need 13 more lines

s2 = pd.Series(np.full(df_master.shape[1],np.nan), index=df_master.columns) #empty row

for i in range(0,13):
    df_master=df_master.append(s2,ignore_index=True)
    
df_master.shape

(13, 59)


(130, 85)

In [102]:
for i in range(0,13):
    df_master.loc[117+i,'YEAR']=block.loc[i,'YEAR']
    df_master.loc[117+i,'CG']=block.loc[i,'CG']
    for j in temp:
        df_master.loc[117+i,j]=block.loc[i,j]
# Quick check
df_master.loc[108:130,['YEAR','CG']+temp]

# Note that row 114 is empty because we don't have data for district 6 in that year.

,YEAR,CG,AGE0-17,AGE18-34,AGE35-49,AGE50-69,AGE70-100,INC0-10k,INC10k-20k,INC20k-40k,...,RACE_4,RACE_5,RACE_6,RACE_7,RACE_8,RACE_9,SEX_1,SEX_2,URBAN_1.0,URBAN_2.0
108,1990.0,0.0,0.282009,0.228656,0.196611,0.185244,0.107480,0.438309,0.275503,0.226145,...,0.000257,0.000257,0.004009,0.003238,0.0,0.0,0.498703,0.501297,0.788749,0.211251
109,1990.0,1.0,0.284517,0.273269,0.218571,0.157918,0.065726,0.321531,0.251994,0.290271,...,0.003486,0.002179,0.022004,0.001307,0.0,0.0,0.491068,0.508932,0.458170,0.541830
110,1990.0,2.0,0.289417,0.296760,0.237689,0.130562,0.045572,0.289375,0.225156,0.326719,...,0.002129,0.001277,0.011816,0.003194,0.0,0.0,0.501810,0.498190,0.154247,0.845753
111,1990.0,3.0,0.291148,0.298377,0.235810,0.139235,0.035431,0.296156,0.256335,0.336321,...,0.001634,0.000117,0.007935,0.002450,0.0,0.0,0.494049,0.505951,0.116686,0.883314
112,1990.0,4.0,0.254285,0.292675,0.222113,0.158506,0.072422,0.327307,0.247357,0.290879,...,0.003713,0.001109,0.033992,0.010704,0.0,0.0,0.484812,0.515188,0.093202,0.906798
113,1990.0,5.0,0.226230,0.305463,0.223363,0.164353,0.080592,0.310937,0.253444,0.289766,...,0.003156,0.001608,0.023547,0.004882,0.0,0.0,0.484907,0.515093,0.023160,0.976840
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,1990.0,7.0,0.309762,0.267925,0.197493,0.156219,0.068601,0.413519,0.281330,0.235193,...,0.000000,0.000139,0.002368,0.000836,0.0,0.0,0.503483,0.496517,0.682502,0.317498
116,1990.0,8.0,0.253007,0.224567,0.216922,0.195209,0.110296,0.447557,0.241430,0.249745,...,0.000202,0.000000,0.004536,0.001411,0.0,0.0,0.488560,0.511440,0.505493,0.494507
117,1850.0,0.0,0.481481,0.314815,0.185185,0.018519,0.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.517241,0.482759,1.000000,0.000000


In [103]:
# Looks good. Let us also add the general turnout for these older years.
df_gen=pd.read_csv('../Data Sets/general_general.csv')
df_prim=pd.read_csv('../Data Sets/general_primary.csv')
df_prim.head()

# Let us fix the column names and delete the first row and last
df_gen.columns=['Year', 'Eligible_voters','Number_voters','Per_turnout','EDR','PerEDR']
df_prim.columns=['Year', 'Eligible_voters','Number_voters','Per_turnout','EDR','PerEDR']
df_gen= df_gen.drop([0,35])
df_prim=df_prim.drop([0,35])
df_gen.head()

df_gen['Year']=pd.to_numeric(df_gen['Year'])
df_prim['Year']=pd.to_numeric(df_prim['Year'])

df_gen['Eligible_voters']=pd.to_numeric(df_gen['Eligible_voters'].str.replace(',',''))
df_gen['Per_turnout']=pd.to_numeric(df_gen['Per_turnout'].str.rstrip('%'))

# Let us save these two data sets
df_gen.to_csv('df_general.csv')
df_prim.to_csv('df_primary.csv')

#Add general election turnout for older years
for i in range(126,130):
    row=df_gen[df_gen['Year']==df_master.loc[i,'YEAR']].iloc[0]
    row2=df_prim[df_prim['Year']==df_master.loc[i,'YEAR']].iloc[0]
    df_master.loc[i,'gen_pt']=row['Per_turnout']
    df_master.loc[i,'gen_el']=row['Eligible_voters']
    df_master.loc[i,'prim_pt']=row2['Per_turnout']
    df_master.loc[i,'prim_el']=row2['Eligible_voters'] 
    
# Fill in the missing 1990 congresional district
df_master.loc[114,['YEAR','CG']]=[1990,6]
df_master.loc[108:116,:]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,congtot_pt,congir_pt,congdfl_pt,sentot_pt,senir_pt,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot
108,1990.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,1990.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,1990.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,1990.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,1990.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,1990.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,1990.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,1990.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,1990.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now let us add the years 2000-2016


In [104]:
#Quick check
df_master.iloc[27:36]

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,congtot_pt,congir_pt,congdfl_pt,sentot_pt,senir_pt,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot
27,2000.0,0.0,2383506.0,993371.0,1234204.0,155931.0,2469422.0,1047474.0,1181553.0,240395.0,...,1.000000,0.416769,0.517810,1.036046,0.424178,0.478474,0.0,NaN,NaN,74853.4752
28,2000.0,1.0,283221.0,159835.0,117946.0,5440.0,292119.0,132346.0,135086.0,24687.0,...,0.118825,0.564347,0.416445,0.122559,0.453055,0.462435,NaN,NaN,NaN,NaN
29,2000.0,2.0,290799.0,139316.0,138969.0,12514.0,297993.0,148246.0,124316.0,25431.0,...,0.122005,0.479080,0.477887,0.125023,0.497481,0.417178,NaN,NaN,NaN,NaN
30,2000.0,3.0,330603.0,217933.0,101106.0,11564.0,345180.0,160770.0,150111.0,34299.0,...,0.138704,0.659198,0.305823,0.144820,0.465757,0.434877,NaN,NaN,NaN,NaN
31,2000.0,4.0,272724.0,83852.0,130403.0,58469.0,282725.0,96020.0,154687.0,32018.0,...,0.114421,0.307461,0.478150,0.118617,0.339623,0.547129,NaN,NaN,NaN,NaN
32,2000.0,5.0,256054.0,62470.0,173575.0,20009.0,277545.0,72124.0,165409.0,40012.0,...,0.107427,0.243972,0.677884,0.116444,0.259864,0.595972,NaN,NaN,NaN,NaN
33,2000.0,6.0,364408.0,170900.0,176340.0,17168.0,369641.0,170883.0,162743.0,36015.0,...,0.152887,0.468980,0.483908,0.155083,0.462294,0.440273,NaN,NaN,NaN,NaN
34,2000.0,7.0,276046.0,79175.0,185771.0,11100.0,282545.0,136151.0,124100.0,22294.0,...,0.115815,0.286818,0.672971,0.118542,0.481874,0.439222,NaN,NaN,NaN,NaN
35,2000.0,8.0,309651.0,79890.0,210094.0,19667.0,321674.0,130934.0,165101.0,25639.0,...,0.129914,0.258000,0.678486,0.134958,0.407039,0.513256,NaN,NaN,NaN,NaN


In [105]:
df_cen['YEAR']=pd.to_numeric(df_cen['YEAR'])
df_cen['CG']=pd.to_numeric(df_cen['CG'])

for i in range(27,108,9): #9 rows for each year
    block=df_cen[df_cen['YEAR']==df_master.loc[i,'YEAR']] # find year block
    for j in range(0,9):
        row=block[block['CG']==j].iloc[0] #find congressional district row
        for k in temp: #add data
            df_master.loc[i+j,k]=row[k]
df_master.iloc[27:36,:]
            
# Looks good.

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,congtot_pt,congir_pt,congdfl_pt,sentot_pt,senir_pt,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot
27,2000.0,0.0,2383506.0,993371.0,1234204.0,155931.0,2469422.0,1047474.0,1181553.0,240395.0,...,1.000000,0.416769,0.517810,1.036046,0.424178,0.478474,0.0,NaN,NaN,74853.4752
28,2000.0,1.0,283221.0,159835.0,117946.0,5440.0,292119.0,132346.0,135086.0,24687.0,...,0.118825,0.564347,0.416445,0.122559,0.453055,0.462435,NaN,NaN,NaN,NaN
29,2000.0,2.0,290799.0,139316.0,138969.0,12514.0,297993.0,148246.0,124316.0,25431.0,...,0.122005,0.479080,0.477887,0.125023,0.497481,0.417178,NaN,NaN,NaN,NaN
30,2000.0,3.0,330603.0,217933.0,101106.0,11564.0,345180.0,160770.0,150111.0,34299.0,...,0.138704,0.659198,0.305823,0.144820,0.465757,0.434877,NaN,NaN,NaN,NaN
31,2000.0,4.0,272724.0,83852.0,130403.0,58469.0,282725.0,96020.0,154687.0,32018.0,...,0.114421,0.307461,0.478150,0.118617,0.339623,0.547129,NaN,NaN,NaN,NaN
32,2000.0,5.0,256054.0,62470.0,173575.0,20009.0,277545.0,72124.0,165409.0,40012.0,...,0.107427,0.243972,0.677884,0.116444,0.259864,0.595972,NaN,NaN,NaN,NaN
33,2000.0,6.0,364408.0,170900.0,176340.0,17168.0,369641.0,170883.0,162743.0,36015.0,...,0.152887,0.468980,0.483908,0.155083,0.462294,0.440273,NaN,NaN,NaN,NaN
34,2000.0,7.0,276046.0,79175.0,185771.0,11100.0,282545.0,136151.0,124100.0,22294.0,...,0.115815,0.286818,0.672971,0.118542,0.481874,0.439222,NaN,NaN,NaN,NaN
35,2000.0,8.0,309651.0,79890.0,210094.0,19667.0,321674.0,130934.0,165101.0,25639.0,...,0.129914,0.258000,0.678486,0.134958,0.407039,0.513256,NaN,NaN,NaN,NaN


In [106]:
# A final sort by year and congressional district
df_master=df_master.sort_values(by=['YEAR','CG']).reset_index()
df_master.to_csv('df_master2.csv')

### State-wide level data
Since my task was to train a model predicting on _state-wide_ turnout, I focus on state-wide level data from now on.

In [107]:
# Let us start by loading the data
df_master=pd.read_csv('df_master_wed.csv')
df_master.head()

#Now let us separate the state-wide data
df_state=df_master[df_master['CG']==0]
df_state.head()

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,congtot_pt,congir_pt,congdfl_pt,sentot_pt,senir_pt,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot
0,1850,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1860,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1870,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1880,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1900,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# Next, let us add weather data as well as the categorical variable for presidential or non-presidential

df_state['Pres']=np.full(df_state.shape[0],np.nan)
df_state['high']=np.full(df_state.shape[0],np.nan)
df_state['low']=np.full(df_state.shape[0],np.nan)
df_state['precipitation']=np.full(df_state.shape[0],np.nan)
df_state['wind']=np.full(df_state.shape[0],np.nan)

In [111]:
#Reset index
#df_state=df_state.reset_index()

df_state=df_state.drop(columns=['index'])
df_state.head(30)

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot,Pres,high,low,precipitation,wind
0,1850,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1860,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1870,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1880,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1900,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1910,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1920,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1930,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1940,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1950,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
# Let us focus further on the years we want to use
df_state2=df_state.iloc[[9,10,11,12,13]+list(range(17,26)),:]
#df_state2=df_state2.drop(columns=['level_0','Unnamed: 0'])
df_state2=df_state2.reset_index()
df_state2=df_state2.drop(columns=['index'])
df_state2.head()

,YEAR,CG,CONGTOT,CONGIR,CONGDFL,CONGTH,SENTOT,SENIR,SENDFL,SENTH,...,sendfl_pt,govtot_pt,govir_pt,govdfl_pt,dif_tot,Pres,high,low,precipitation,wind
0,1950,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1960,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1970,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
# Now let us enter the new data (weather etc)
df_state2['high']=[48,45,43,50,45,48,36,48,61,71,56,44,53,59]
df_state2['low']=[37,37,34,25,30,30,30,36,44,51,34,38,37,39]
df_state2['precipitation']=[0,0,0.53,0.02,0,0.06,0.03,0.06,0,0,0,0.31,0,0]
df_state2['wind']=[10,18,22,21,16,24,12,17,15,25,17,20,29,17]

df_state2['YEAR']=pd.to_numeric(df_state2['YEAR'])

#Finally, let us add the categorical variable keeping track of whether the election was presidential or not
for i in range(0,df_state2.shape[0]):
    if df_state2.loc[i,'YEAR']%4==0:
        df_state2.loc[i,'pres']=1
    else:
        df_state2.loc[i,'pres']=0
# let us save this one too
df_state2.to_csv('df_state.csv')

#### Some further cleaning

In [114]:
print(df_state2.shape)
list(df_state2.columns.values)

print(df_state2.columns.get_loc('AGE0-17'),' to ',df_state2.columns.get_loc('EMPSTAT_3.0'))
print(df_state2.columns.get_loc('HISPAN_0'),' to ',df_state2.columns.get_loc('MARST_6.0'))
print(df_state2.columns.get_loc('RACE_1'),' to ',df_state2.columns.get_loc('SEX_2'))
print(df_state2.columns.get_loc('high'), ' to ', df_state2.columns.get_loc('pres'))

(14, 91)
14  to  39
42  to  52
58  to  68
86  to  90


In [115]:
names=list(range(14,40))+list(range(42,53))+list(range(58,69))+list(range(86,91))
names=df_state2.columns[names]

df=df_state2[['YEAR']+list(names)+['prim_pt','gen_pt']]

# For some reason, there are some missing values for 1990, let us fix those
df.loc[4,'gen_pt']=58.76
df.loc[4,'prim_pt']= 24.61

df.head(14)

,YEAR,AGE0-17,AGE18-34,AGE35-49,AGE50-69,AGE70-100,INC0-10k,INC10k-20k,INC20k-40k,INC40k-60k,...,RACE_9,SEX_1,SEX_2,high,low,precipitation,wind,pres,prim_pt,gen_pt
0,1950,0.297810,0.249623,0.193565,0.202865,0.056138,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.503318,0.496682,48,37,0.00,10,0.0,31.06%,56.84
1,1960,0.361409,0.204856,0.183872,0.185346,0.064517,0.963312,0.029216,0.007472,0.000000,...,0.000000,0.495410,0.504590,45,37,0.00,18,1.0,30.01%,79.39
2,1970,0.352826,0.246394,0.159213,0.168829,0.072739,0.837427,0.136298,0.021257,0.005018,...,0.000000,0.484921,0.515079,43,34,0.53,22,0.0,32.26%,62.38
3,1980,0.278443,0.305635,0.158309,0.175342,0.082271,0.575629,0.268393,0.132573,0.014349,...,0.000000,0.488968,0.511032,50,25,0.02,21,1.0,13.78%,72.14
4,1990,0.256264,0.283639,0.208312,0.162653,0.089132,0.375774,0.256768,0.265430,0.066446,...,0.000000,0.490965,0.509035,45,30,0.00,16,0.0,24.61,58.76
5,2000,0.252168,0.234014,0.245508,0.177051,0.091259,0.240205,0.197448,0.323408,0.136962,...,0.001064,0.494689,0.505311,48,30,0.06,24,1.0,17.19,70.11
6,2002,0.256116,0.206406,0.248813,0.195660,0.093005,0.223972,0.181371,0.311775,0.157329,...,0.001134,0.491880,0.508120,36,30,0.03,12,0.0,15.23,64.89
7,2004,0.237914,0.231197,0.243956,0.203409,0.083523,0.223805,0.179948,0.305835,0.157301,...,0.000899,0.496082,0.503918,48,36,0.06,17,1.0,7.73,78.77
8,2006,0.231908,0.233324,0.232045,0.212188,0.090535,0.222246,0.180971,0.285945,0.164764,...,0.001269,0.497048,0.502952,61,44,0.00,15,0.0,14.02,60.47
9,2008,0.229741,0.232569,0.221081,0.228030,0.088579,0.215141,0.165183,0.277027,0.169232,...,0.001623,0.497344,0.502656,71,51,0.00,25,1.0,11.22,78.11


In [116]:
# Check whether there are any zero entries
df.isna().sum()

YEAR             0
AGE0-17          0
AGE18-34         0
AGE35-49         0
AGE50-69         0
AGE70-100        0
INC0-10k         0
INC10k-20k       0
INC20k-40k       0
INC40k-60k       0
INC60k-100k      0
INC>100k         0
EDUC_0.0         0
EDUC_1.0         0
EDUC_10.0        0
EDUC_11.0        0
EDUC_2.0         0
EDUC_3.0         0
EDUC_4.0         0
EDUC_5.0         0
EDUC_6.0         0
EDUC_7.0         0
EDUC_8.0         0
EMPSTAT_0.0      0
EMPSTAT_1.0      0
EMPSTAT_2.0      0
EMPSTAT_3.0      0
HISPAN_0         0
HISPAN_1         0
HISPAN_2         0
HISPAN_3         0
HISPAN_4         0
MARST_1.0        0
MARST_2.0        0
MARST_3.0        0
MARST_4.0        0
MARST_5.0        0
MARST_6.0        0
RACE_1           0
RACE_2           0
RACE_3           0
RACE_4           0
RACE_5           0
RACE_6           0
RACE_7           0
RACE_8           0
RACE_9           0
SEX_1            0
SEX_2            0
high             0
low              0
precipitation    0
wind        

In [117]:
# Save this data set just in case
df.to_csv('df_state_ml.csv')